In [1]:
import numpy as np
import pandas as pd
import matlab.engine
import yaml
import time
import importlib
import pathlib
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
from scipy.stats import pearsonr
from loguru import logger
from Entropy import ApproximateEntropy
from functools import partial, wraps
from itertools import product
import os
from typing import Union
from numbers import Number
from utilities import ZScore

In [87]:
import importlib
import Information
importlib.reload(Information)

<module 'Information' from '/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Operations/Information.py'>

In [3]:
def compare_outputs(outputs, exclude_keys=None):
    """
    Compare MATLAB and Python feature outputs, computing relative errors
    and Pearson correlations.

    Parameters
    ----------
    outputs : dict
        Nested dictionary of feature values with structure:
        {
            feature_name: {
                ts_id: {
                    'matlab': scalar_or_dict,
                    'python': scalar_or_dict
                }
            }
        }
    exclude_keys : set or list, optional
        Set or list of keys (feature names or feature.subkey) to exclude from comparison.

    Returns
    -------
    results : dict
        Dictionary mapping feature (or subfeature) names to correlation and stats:
        {
            'feature.subkey': {
                'r': float,
                'pval': float,
                'res_py': ndarray,
                'res_matlab': ndarray,
                'max_rel_err': float
            }
        }
    """
    if exclude_keys is None:
        exclude_keys = set()
    else:
        exclude_keys = set(exclude_keys)

    flat = {}
    for feat, ts_dict in outputs.items():
        for ts, run in ts_dict.items():
            ml = run['matlab']
            py = run['python']
            if isinstance(ml, dict) and isinstance(py, dict):
                for k, mlv in ml.items():
                    slot = f"{feat}.{k}"
                    if slot in exclude_keys or k not in py:
                        continue
                    pyv = py[k]
                    flat.setdefault(slot, {})[ts] = (mlv, pyv)
            elif isinstance(ml, Number) and isinstance(py, Number):
                if feat in exclude_keys:
                    continue
                flat.setdefault(feat, {})[ts] = (ml, py)
            else:
                raise ValueError(f"Feature {feat}@{ts} is neither both scalars nor both dicts.")

    results = {}
    for slot, tsmap in flat.items():
        ml_vals, py_vals = [], []
        rel_errors = []

        for ts, (mlv, pyv) in tsmap.items():
            ml_vals.append(mlv)
            py_vals.append(pyv)

            both_finite = np.isfinite(mlv) and np.isfinite(pyv)
            both_nan = np.isnan(mlv) and np.isnan(pyv)
            both_posinf = (mlv == np.inf) and (pyv == np.inf)
            both_neginf = (mlv == -np.inf) and (pyv == -np.inf)

            if both_finite:
                if mlv == 0:
                    rel_err = np.nan
                else:
                    rel_err = abs(mlv - pyv) / abs(mlv) * 100
                    rel_errors.append(rel_err)
                print(f"[{slot} | ts={ts}]  RelErr% = {rel_err:.2f}")
            elif both_nan or both_posinf or both_neginf:
                print(f"[{slot} | ts={ts}]  RelErr% = MATCH (both non-finite)")
            else:
                print(f"[{slot} | ts={ts}]  RelErr% = NaN (mismatch in finiteness)")

        ml_arr = np.array(ml_vals, dtype=float)
        py_arr = np.array(py_vals, dtype=float)
        finite_mask = np.isfinite(ml_arr) & np.isfinite(py_arr)

        if finite_mask.sum() > 1 and ml_arr[finite_mask].std() and py_arr[finite_mask].std():
            r, p = pearsonr(ml_arr[finite_mask], py_arr[finite_mask])
        else:
            r, p = np.nan, np.nan

        max_rel_err = np.nanmax(rel_errors) if rel_errors else np.nan

        results[slot] = {
            'r': r,
            'pval': p,
            'res_py': py_arr,
            'res_matlab': ml_arr,
            'max_rel_err': max_rel_err
        }

    return results

In [90]:
def compare_outputs(outputs, exclude_keys=None):
    """
    Compare MATLAB and Python feature outputs, computing relative errors
    and Pearson correlations.

    Parameters
    ----------
    outputs : dict
        Nested dictionary of feature values with structure:
        {
            feature_name: {
                ts_id: {
                    'matlab': scalar_or_dict,
                    'python': scalar_or_dict
                }
            }
        }
    exclude_keys : set or list, optional
        Set or list of keys (feature names or feature.subkey) to exclude from comparison.

    Returns
    -------
    results : dict
        Dictionary mapping feature (or subfeature) names to correlation and stats:
        {
            'feature.subkey': {
                'r': float,
                'pval': float,
                'res_py': ndarray,
                'res_matlab': ndarray,
                'max_rel_err': float
            }
        }
    """
    if exclude_keys is None:
        exclude_keys = set()
    else:
        exclude_keys = set(exclude_keys)

    flat = {}
    for feat, ts_dict in outputs.items():
        for ts, run in ts_dict.items():
            ml = run['matlab']
            py = run['python']
            if isinstance(ml, dict) and isinstance(py, dict):
                for k, mlv in ml.items():
                    slot = f"{feat}.{k}"
                    if slot in exclude_keys or k not in py:
                        continue
                    pyv = py[k]
                    flat.setdefault(slot, {})[ts] = (mlv, pyv)
            elif isinstance(ml, Number) and isinstance(py, Number):
                if feat in exclude_keys:
                    continue
                flat.setdefault(feat, {})[ts] = (ml, py)
            else:
                raise ValueError(f"Feature {feat}@{ts} is neither both scalars nor both dicts.")

    results = {}
    for slot, tsmap in flat.items():
        ml_vals, py_vals = [], []
        rel_errors = []

        for ts, (mlv, pyv) in tsmap.items():
            ml_vals.append(mlv)
            py_vals.append(pyv)

            both_finite = np.isfinite(mlv) and np.isfinite(pyv)
            both_nan = np.isnan(mlv) and np.isnan(pyv)
            both_posinf = (mlv == np.inf) and (pyv == np.inf)
            both_neginf = (mlv == -np.inf) and (pyv == -np.inf)

            if both_finite:
                if mlv == 0:
                    rel_err = np.nan
                else:
                    rel_err = abs(mlv - pyv) / abs(mlv) * 100
                    rel_errors.append(rel_err)
                print(f"[{slot} | ts={ts}]  RelErr% = {rel_err:.2f}")
            elif both_nan or both_posinf or both_neginf:
                print(f"[{slot} | ts={ts}]  RelErr% = MATCH (both non-finite)")
            else:
                print(f"[{slot} | ts={ts}]  RelErr% = NaN (mismatch in finiteness)")

        ml_arr = np.array(ml_vals, dtype=float)
        py_arr = np.array(py_vals, dtype=float)
        finite_mask = np.isfinite(ml_arr) & np.isfinite(py_arr)

        if finite_mask.sum() > 1 and ml_arr[finite_mask].std() and py_arr[finite_mask].std():
            r, p = pearsonr(ml_arr[finite_mask], py_arr[finite_mask])
        else:
            r, p = np.nan, np.nan

        max_rel_err = np.nanmax(rel_errors) if rel_errors else np.nan

        results[slot] = {
            'r': r,
            'pval': p,
            'res_py': py_arr,
            'res_matlab': ml_arr,
            'max_rel_err': max_rel_err
        }

    return results

In [91]:
empirical1000 = []
with open("../../../empirical1000/hctsa_timeseries-data.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        # Convert each element from string to float (or int if appropriate)
        try:
            time_series = [float(value) for value in row if value != '']
            empirical1000.append(time_series)
        except ValueError as e:
            print(f"Skipping row due to conversion error: {row}")
            continue

In [89]:
def load_yaml(file):
    print(f"Loading configuration file: {file.split('/')[-1]}")
    funcs = {}
    with open(file) as f:
        yf = yaml.load(f, Loader=yaml.FullLoader)

    for module_name in yf:
        print(f"\n*** Importing module {module_name} *** \n")
        module = importlib.import_module(module_name)
        for function_name in yf[module_name]:
            # Get the function's configuration dictionary
            function_config = yf[module_name][function_name]
            # If no configs section exists or if it's empty, use a list with single empty dict
            if ('configs' not in function_config or function_config.get('configs') is None or 
                function_config.get('configs') == []):
                configs = [{}]
            else:
                configs = function_config.get('configs', [{}])

            for params in configs:
                # Handle the case where params is None
                if params is None:
                    params = {}
                    
                zscore_first = params.pop("zscore", False)
                param_keys, param_vals = zip(*params.items()) if params else ([], [])
                
                param_combinations = [dict(zip(param_keys, values)) 
                                   for values in product(*[v if isinstance(v, list) 
                                                        else [v] for v in param_vals])]
                
                # If no parameter combinations were generated, add empty dict
                if not param_combinations:
                    param_combinations = [{}]
                
                # create a function for each parameter combination
                for param_set in param_combinations:
                    feature_name = (f"{module_name}_{function_name}_" + 
                                  "_".join(f"{v}" for k, v in param_set.items())
                                  if param_set else f"{module_name}_{function_name}")
                    if not zscore_first:
                        feature_name += "_raw"
                    
                    print(f"Adding operation {feature_name} with params {param_set} "
                          f"(Z-score={zscore_first})")
                    
                    base_func = partial(getattr(module, function_name), **param_set)
                    if zscore_first:
                        base_func = zscore_decorator(base_func)
                    
                    # return the MATLAB callable corresponding to the python implementation for direct comparison
                    # make sure to check whethe the data needs to be zscored when calling the MATLAB func, cannot be wrapped as it is not a python function
                    # so needs to be done manually when calling the function.
                    hctsa_name = function_config.get('hctsa_name')
                    hctsa_callable = eval(f"eng.{hctsa_name}")

                    # keep ordered args only for testing YAML otherwise bloats
                    funcs[feature_name] = {'callable': base_func, 'params': param_set, 'hctsa_name': function_config.get('hctsa_name'), 
                                           'matlab_callable': hctsa_callable, 'isZscore': zscore_first, 'ordered_args': function_config.get('ordered_args')}
                    
    return funcs

In [26]:
def eval_comparison(yaml, data):
    func_dict = load_yaml(yaml)
    func_res = dict()
    for func in func_dict:
        print(f"Evalutating {func}")
        f = func_dict[func]
        python_func = f['callable']
        matlab_func = f['matlab_callable']
        hctsa_name = f['hctsa_name']
        isZscore = f['isZscore']
        params = f['params']
        ordered_args = []
        if params:
            order = f['ordered_args']
            ordered_args = [params[k] for k in order]
            
        print(f"Comparing to {hctsa_name}")
        res = dict()
        for i in range(len(data)):
            x = np.array(data[i])
            matlab_eval = matlab_func(ZScore(matlab.double(x)), *ordered_args) if isZscore else matlab_func(matlab.double(x), matlab.double(list(range(1, 11))), matlab.double(ordered_args[0]), matlab.double(ordered_args[1]), matlab.double(ordered_args[2]))
            python_eval = python_func(x)
            res[i] = {'matlab': matlab_eval, 'python': python_eval}
        func_res[func] = res
    return func_res

In [8]:
eng = matlab.engine.start_matlab()

In [19]:
proj_root = pathlib.Path("/Users/jmoo2880/Documents/hctsa")
jar_path = "/Users/jmoo2880/Documents/hctsa/Toolboxes/infodynamics-dist/infodynamics.jar"
eng.eval(f"javaaddpath('{jar_path}')", nargout=0)
eng.addpath(eng.genpath(str(proj_root)), nargout=0)
def zscore_decorator(func):
    @wraps(func)
    def wrapper(y, *args, **kwargs):
        y = ZScore(y)
        return func(y, *args, **kwargs)
    return wrapper

def range_constructor(loader, node):
    start, end = loader.construct_sequence(node)
    return list(range(start, end+1))
yaml.add_constructor("!range", range_constructor)

In [39]:
Information.AutoMutualInfo(empirical1000[0], 'ac', 'kraskov1')

np.float64(0.00028166446528032907)

In [99]:
Information.AutoMutualInfoStats(empirical1000[198], 2, 'kraskov1')

{'ami1': np.float64(2.6228027748949163),
 'ami2': np.float64(0.8852533356686658),
 'mami': np.float64(1.754028055281791),
 'stdami': np.float64(1.2286329911237646),
 'pextrema': 0.0,
 'fmmi': 2,
 'pmaxima': 0.0,
 'modeperiodmax': nan,
 'pmodeperiodmax': nan,
 'pminima': 0.0,
 'modeperiodmin': nan,
 'pmodeperiodmin': nan,
 'pcrossmean': np.float64(1.0),
 'pcrossmedian': np.float64(1.0),
 'pcrossq10': np.float64(0.0),
 'pcrossq90': np.float64(0.0),
 'amiac1': np.float64(-0.5)}

In [71]:
Information.FirstMin(empirical1000[97], 'mi-hist')

4

In [31]:
funcs = load_yaml("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml")

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_AutoMutualInfoStats_20_kraskov1_4 with params {'maxTau': 20, 'estMethod': 'kraskov1', 'extraParam': '4'} (Z-score=True)


In [32]:
res = eval_comparison("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml", empirical1000)

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_AutoMutualInfoStats_20_kraskov1_4 with params {'maxTau': 20, 'estMethod': 'kraskov1', 'extraParam': '4'} (Z-score=True)
Evalutating Information_AutoMutualInfoStats_20_kraskov1_4
Comparing to IN_AutoMutualInfoStats
> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0080

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0087

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0056

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0228

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0317

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0213

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutualInfoStats (line 76)

ans =

   -0.0123

> In IN_AutoMutualInfo (line 72)
In IN_AutoMutual

In [33]:
res_res = compare_outputs(res)

[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=0]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=1]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=2]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=3]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=4]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=5]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=6]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=7]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=8]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=9]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=10]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=11]  RelErr% = 0.00
[Information_AutoMutualInfoStats_20_kraskov1_4.ami1 | ts=12]  RelErr% = 0.00
[Informat

In [30]:
[res_res[p]['r'] for p in res_res]

[np.float64(0.9999999999999999),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(0.9999999999999997),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999998),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(0.9999999999999998),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(0.9999999999999998),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999998),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999998),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.9999999999999999),
 np.float64(1.0),
 np.

In [34]:
funcs2 = load_yaml("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml")

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_FirstMin_ac_None_None with params {'minWhat': 'ac', 'extraParam': None, 'minNotMax': None} (Z-score=True)
Adding operation Information_FirstMin_mi-gaussian_None_None with params {'minWhat': 'mi-gaussian', 'extraParam': None, 'minNotMax': None} (Z-score=True)


In [39]:
eng.CO_FirstMin(ZScore(matlab.double(empirical1000[0])), 'ac')

2.0

In [40]:
Information.FirstMin(ZScore(empirical1000[0]), 'ac')

2

In [43]:
def eval_comparison_firstmin(yaml, data):
    func_dict = load_yaml(yaml)
    func_res = dict()
    for func in func_dict:
        print(f"Evalutating {func}")
        f = func_dict[func]
        python_func = f['callable']
        matlab_func = f['matlab_callable']
        hctsa_name = f['hctsa_name']
        isZscore = f['isZscore']
        params = f['params']
        ordered_args = []
        if params:
            order = f['ordered_args']
            ordered_args = [params[k] for k in order]
            
        print(f"Comparing to {hctsa_name}")
        res = dict()
        for i in range(len(data)):
            x = np.array(data[i])
            matlab_eval = matlab_func(ZScore(matlab.double(x)), *ordered_args) if isZscore else matlab_func(matlab.double(x), *ordered_args)
            python_eval = python_func(x)
            res[i] = {'matlab': matlab_eval, 'python': python_eval}
        func_res[func] = res
    return func_res

In [45]:
res_firstMin = eval_comparison_firstmin("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml", empirical1000)

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_FirstMin_mi-hist_5_ with params {'minWhat': 'mi-hist', 'extraParam': 5, 'minNotMax': ''} (Z-score=True)
Adding operation Information_FirstMin_mi-hist_10_ with params {'minWhat': 'mi-hist', 'extraParam': 10, 'minNotMax': ''} (Z-score=True)
Evalutating Information_FirstMin_mi-hist_5_
Comparing to CO_FirstMin
Evalutating Information_FirstMin_mi-hist_10_
Comparing to CO_FirstMin


In [47]:
comp_outs = compare_outputs(res_firstMin)

[Information_FirstMin_mi-hist_5_ | ts=0]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=1]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=2]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=3]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=4]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=5]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=6]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=7]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=8]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=9]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=10]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=11]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=12]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=13]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=14]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=15]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_ | ts=16]  RelErr% = 0.00
[Information_FirstMin_mi

In [49]:
[comp_outs[p]['r'] for p in comp_outs]

[np.float64(1.0), np.float64(0.9999999999999998)]

In [50]:
res_firstMin = eval_comparison_firstmin("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml", empirical1000)

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_FirstMin_mi-hist_5_False with params {'minWhat': 'mi-hist', 'extraParam': 5, 'minNotMax': False} (Z-score=True)
Adding operation Information_FirstMin_mi-hist_10_False with params {'minWhat': 'mi-hist', 'extraParam': 10, 'minNotMax': False} (Z-score=True)
Evalutating Information_FirstMin_mi-hist_5_False
Comparing to CO_FirstMin
Evalutating Information_FirstMin_mi-hist_10_False
Comparing to CO_FirstMin


In [51]:
comp_outs2 = compare_outputs(res_firstMin)

[Information_FirstMin_mi-hist_5_False | ts=0]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=1]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=2]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=3]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=4]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=5]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=6]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=7]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=8]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=9]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=10]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=11]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=12]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=13]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=14]  RelErr% = 0.00
[Information_FirstMin_mi-hist_5_False | ts=15]  RelErr% = 0.00
[I

In [52]:
[comp_outs2[p]['r'] for p in comp_outs2]

[np.float64(1.0), np.float64(0.9999999999999998)]

In [58]:
idxs = np.random.randint(0, 1000, 50)

In [69]:
res_firstMin3 = eval_comparison_firstmin("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/information.yaml",[empirical1000[i] for i in idxs])

Loading configuration file: information.yaml

*** Importing module Information *** 

Adding operation Information_FirstMin_mi-kraskov1_4_ with params {'minWhat': 'mi-kraskov1', 'extraParam': '4', 'minNotMax': ''} (Z-score=True)
Adding operation Information_FirstMin_mi-kraskov2_4_ with params {'minWhat': 'mi-kraskov2', 'extraParam': '4', 'minNotMax': ''} (Z-score=True)
Evalutating Information_FirstMin_mi-kraskov1_4_
Comparing to CO_FirstMin
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 124)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 124)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 124)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 124)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInf

In [71]:
compare_outputs(res_firstMin3)

[Information_FirstMin_mi-kraskov1_4_ | ts=0]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=1]  RelErr% = 12.50
[Information_FirstMin_mi-kraskov1_4_ | ts=2]  RelErr% = 60.00
[Information_FirstMin_mi-kraskov1_4_ | ts=3]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=4]  RelErr% = 41.67
[Information_FirstMin_mi-kraskov1_4_ | ts=5]  RelErr% = 30.00
[Information_FirstMin_mi-kraskov1_4_ | ts=6]  RelErr% = 20.59
[Information_FirstMin_mi-kraskov1_4_ | ts=7]  RelErr% = 30.00
[Information_FirstMin_mi-kraskov1_4_ | ts=8]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=9]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=10]  RelErr% = 20.00
[Information_FirstMin_mi-kraskov1_4_ | ts=11]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=12]  RelErr% = 0.00
[Information_FirstMin_mi-kraskov1_4_ | ts=13]  RelErr% = 30.00
[Information_FirstMin_mi-kraskov1_4_ | ts=14]  RelErr% = 19.05
[Information_FirstMin_mi-kraskov1_4_ | ts=15]  RelErr% = 0.00
[Informat

{'Information_FirstMin_mi-kraskov1_4_': {'r': np.float64(0.9817948386367127),
  'pval': np.float64(2.759242049037584e-36),
  'res_py': array([12.,  7.,  2.,  6.,  7.,  7., 41.,  7.,  7., 19., 16.,  4.,  8.,
          7., 17.,  6.,  5., 12.,  2.,  9.,  4.,  9.,  2.,  4., 22.,  3.,
          9., 11.,  2.,  5., 35.,  8.,  8., 16.,  4.,  6.,  9.,  3.,  5.,
          2.,  5., 22., 75.,  3., 23.,  3.,  4., 22., 28.,  5.]),
  'res_matlab': array([12.,  8.,  5.,  6., 12., 10., 34., 10.,  7., 19., 20.,  4.,  8.,
         10., 21.,  6.,  5., 12.,  2., 14.,  4.,  3.,  2.,  3., 16.,  3.,
          9., 12.,  2.,  8., 37.,  8., 10., 16.,  4.,  6.,  9.,  3.,  4.,
          2.,  5., 22., 71.,  3., 23.,  3.,  4., 25., 25.,  4.]),
  'max_rel_err': np.float64(200.0)},
 'Information_FirstMin_mi-kraskov2_4_': {'r': np.float64(0.9999999999999999),
  'pval': np.float64(0.0),
  'res_py': array([14.,  7.,  2.,  6.,  7., 10., 28., 19.,  7., 15., 20.,  5.,  8.,
         10., 21.,  6.,  5., 12.,  3., 14.,  3.,  9

In [ ]:
[Information_FirstMin_mi-kraskov1_4_ | ts=21]  RelErr% = 200.00

In [73]:
res_firstMin3['Information_FirstMin_mi-kraskov1_4_'][21]

{'matlab': 3.0, 'python': 9}

In [88]:
Information.FirstMin(ZScore(empirical1000[21]), 'mi-kraskov1', '4')

[2.02871697 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

6

In [ ]:
out = eng.CO_FirstMin(matlab.double(ZScore(empirical1000[21])), 'mi-kraskov1', '4')

> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)
In CO_FirstMin>@(x)IN_AutoMutualInfo(y,x,'kraskov1') (line 73)
In CO_FirstMin (line 99)
> In IN_AutoMutualInfo (line 72)

In [84]:
out

11.0